In [21]:
# Importing dependencies
import pandas as pd
import requests

# Reading the CSV

In [22]:
# Reading CSV file
df = pd.read_csv('sac_crime.csv')

df.head()

,Beat,Description,Grid,Occurence_Date,Offense_Category,Offense_Code,Offense_Ext,Police_District,Record_ID,Day_of_the_Week,Month,Year,Hour,Encoded_Beat
0,3B,487(A) PC GRAND THEFT >$950,748,"Wed 01-01-2020, 01:15:00 AM",LARCENY,2303,1,3,1429889,3,1,2020,1,7
1,3B,484 PC PETTY THEFT,748,"Wed 01-01-2020, 01:30:00 AM",LARCENY,2399,2,3,1429894,3,1,2020,1,7
2,3A,459 PC BURG VEHICLE,725,"Wed 01-01-2020, 10:40:00 AM",BURGLARY,2299,01,3,1429896,3,1,2020,10,6
3,4A,459.5 BURGLARY SHOPLIFT < $950,965,"Wed 01-01-2020, 02:30:00 PM",BURGLARY,2205,3,4,1429900,3,1,2020,14,9
4,4A,459.5 BURGLARY SHOPLIFT < $950,965,"Wed 01-01-2020, 02:10:00 PM",BURGLARY,2205,3,4,1429902,3,1,2020,14,9


In [23]:
# Setting features
selected_features = df[['Offense_Category','Encoded_Beat','Police_District','Grid',
                        'Day_of_the_Week','Month','Year','Hour']]
selected_features.head()

,Offense_Category,Encoded_Beat,Police_District,Grid,Day_of_the_Week,Month,Year,Hour
0,LARCENY,7,3,748,3,1,2020,1
1,LARCENY,7,3,748,3,1,2020,1
2,BURGLARY,6,3,725,3,1,2020,10
3,BURGLARY,9,4,965,3,1,2020,14
4,BURGLARY,9,4,965,3,1,2020,14


# Data Preprocessing

In [24]:
# Assigning X and y values
X = selected_features.drop('Offense_Category', axis=1)
y = selected_features['Offense_Category']

print(X.shape, y.shape)

(5835, 7) (5835,)


In [25]:
# Importing dependencies
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [26]:
# Creating a MinMaxScaler model and fitting it to the training data
X_scaler = MinMaxScaler().fit(X_train)

# Transform the training and testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Creating a LabelEncoder model and fitting it to y training data
label_encoder = LabelEncoder()
label_encoder.fit(y)

# Encoding y data
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# One-hot encoding y
one_hot_y_train = to_categorical(encoded_y_train)
one_hot_y_test = to_categorical(encoded_y_test)
one_hot_y_train.shape

(4376, 10)

# Building a Deep Neural Network Model

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Creating an empty sequential model
deep_model = Sequential()

In [28]:
# Adding layers
deep_model.add(Dense(units=100, activation="relu", input_dim=7))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=10, activation='softmax'))

In [29]:
deep_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               800       
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1010      
Total params: 11,910
Trainable params: 11,910
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compiling the model
deep_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [31]:
# Using the training data to fit and train the deep model
deep_model.fit(X_train_scaled, one_hot_y_train, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
137/137 - 2s - loss: 1.4206 - accuracy: 0.4413
Epoch 2/100
137/137 - 2s - loss: 1.2550 - accuracy: 0.4456
Epoch 3/100
137/137 - 2s - loss: 1.2425 - accuracy: 0.4484
Epoch 4/100
137/137 - 2s - loss: 1.2366 - accuracy: 0.4691
Epoch 5/100
137/137 - 2s - loss: 1.2314 - accuracy: 0.4673
Epoch 6/100
137/137 - 1s - loss: 1.2297 - accuracy: 0.4698
Epoch 7/100
137/137 - 1s - loss: 1.2283 - accuracy: 0.4714
Epoch 8/100
137/137 - 1s - loss: 1.2270 - accuracy: 0.4728
Epoch 9/100
137/137 - 1s - loss: 1.2258 - accuracy: 0.4767
Epoch 10/100
137/137 - 1s - loss: 1.2249 - accuracy: 0.4842
Epoch 11/100
137/137 - 2s - loss: 1.2208 - accuracy: 0.4840
Epoch 12/100
137/137 - 2s - loss: 1.2226 - accuracy: 0.4815
Epoch 13/100
137/137 - 2s - loss: 1.2206 - accuracy: 0.4881
Epoch 14/100
137/137 - 1s - loss: 1.2192 - accuracy: 0.4865
Epoch 15/100
137/137 - 1s - loss: 1.2160 - accuracy: 0.4925
Epoch 16/100
137/137 - 1s - loss: 1.2180 - accuracy: 0.4810
Epoch 17/100
137/137 - 1s - loss: 1.2176 - accura

In [12]:
# Evaluating the Deep Model
deep_model_loss, deep_model_accuracy = deep_model.evaluate(X_test_scaled, one_hot_y_test, verbose=5)
print(f'Loss: {deep_model_loss}, Accuracy: {deep_model_accuracy}')

Loss: 1.4028315544128418, Accuracy: 0.4551062285900116
